# Python interface

## Using Analysis, Description class and automatic plotting

For this tutorial we will use the [test data](https://github.com/JaGeo/LobsterPy/tests/test_data) available that can be downloaded from our git repository.

Lets first import the necessary modules

### Basic usage : Analysis, Description

In [ ]:
import os
from lobsterpy.cohp.analyze import Analysis
from lobsterpy.cohp.describe import Description
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Directory of your VASP and Lobster computations
directory = "../../tests/test_data/CdF_comp_range/"

# Initialize Analysis object
analyse = Analysis(
    path_to_poscar=os.path.join(directory, "POSCAR.gz"),
    path_to_icohplist=os.path.join(directory, "ICOHPLIST.lobster.gz"),
    path_to_cohpcar=os.path.join(directory, "COHPCAR.lobster.gz"),
    path_to_charge=os.path.join(directory, "CHARGE.lobster.gz"),
    which_bonds="cation-anion",
)

In [ ]:
# Initialize Description object and get text description of the analysis
describe = Description(analysis_object=analyse)
describe.write_description()

In [ ]:
# Get static plots for relevant bonds
describe.plot_cohps(ylim=[-10, 2], xlim=[-4, 4])

In [ ]:
# Get interactive plots of relevant bonds, 

# Setting label_resolved arg to True will plot each COHP curve seperately, alongside summed COHP for the bonds.

fig = describe.plot_interactive_cohps(label_resolved=True, hide=True)
fig.show(renderer='notebook')

In [ ]:
# Dict summarizing the automaitic analysis results
analyse.condensed_bonding_analysis

In [ ]:
# Dict with bonds identified
analyse.final_dict_bonds

In [ ]:
# Dict with ions and thier co-ordination environments
analyse.final_dict_ions

:::{note}
You can also perform automatic analysis using `COBICAR(ICOBILIST.lobster)` or `COOPCAR(ICOOPLIST.lobster)`. You would need to set `are_cobis`/`are_coops` to `True` depending on type of file you decide to analyze when you initialize Analysis object. And also change the default `noise_cutoff` value to 0.001 or lower as ICOOP and ICOBI have lower values. Below is an example code snippet
:::

```python
analyse = Analysis(
    path_to_poscar=os.path.join(directory, "POSCAR.gz"),
    path_to_icohplist=os.path.join(directory, "ICOBILIST.lobster.gz"),
    path_to_cohpcar=os.path.join(directory, "COBICAR.lobster.gz"),
    path_to_charge=os.path.join(directory, "CHARGE.lobster.gz"),
    which_bonds="cation-anion",
    are_cobis=True,
    noise_cutoff=0.001,
)
```

Rest of the things to access the results are same as above.

### Advanced usage : Analysis, Description

LobsterPy, now also provides the possibility to also perform automatic orbialtwise analysis and plotting of COHPs, COBIs and COOPs. To switch on orbialtwise analysis, one needs to set `orbital_resolved` arg to `True`. By default orbitals contributing 5% or more relative to summed ICOHPs are considered in analysis. One can change this default threshold using `orbital_cutoff` argument. Here we will set this cutoff value to 3%

In [ ]:
analyse = Analysis(
    path_to_poscar=os.path.join(directory, "POSCAR.gz"),
    path_to_icohplist=os.path.join(directory, "ICOHPLIST.lobster.gz"),
    path_to_cohpcar=os.path.join(directory, "COHPCAR.lobster.gz"),
    path_to_charge=os.path.join(directory, "CHARGE.lobster.gz"),
    which_bonds="cation-anion",
    orbital_resolved=True,
    orbital_cutoff=0.03,
)

In [ ]:
# Access the dict summarizing the results including orbital wise analysis data 
analyse.condensed_bonding_analysis

In the above ouput you will see, now a field named `orbital_data` associated to each relevant bond identified. The `orbital_summary_stats` field contains the orbitals that contribute the most to the bonding and antibonding interactions and values are reported there in percent.

:::{note}
You can get plots from orbital resolved anaylsis only when `orbital_resolved` arg to `True` when initializing Analysis object. If this is not done, you will run into errors. Also only interactive plotter will plot the results of orbital resolved anaylsis as static plots will not be much readable. In anycase you can generate static plots if you need to, you will find how to use the plotters below in the tutorial
:::

In [ ]:
# Initialize Description object
describe = Description(analysis_object=analyse)
describe.write_description()

In [ ]:
# Automatic interactive plots
fig = describe.plot_interactive_cohps(orbital_resolved=True, ylim=[-15,5], hide=True)
fig.show(renderer='notebook')

#### Get LOBSTER calculation quality and description

In [ ]:
# Directory to your VASP and Lobster computations
directory = "../../tests/test_data/K3Sb/"

In [ ]:
# Get calculation quality summary dict
calc_quality_K3Sb = Analysis.get_lobster_calc_quality_summary(
            path_to_poscar=os.path.join(directory, "POSCAR.gz"),
            path_to_charge=os.path.join(directory, "CHARGE.lobster.gz"),
            path_to_lobsterin=os.path.join(directory,"lobsterin.gz"),
            path_to_lobsterout=os.path.join(directory,"lobsterout.gz"),
            potcar_symbols=["K_sv", "Sb"], # if POTCAR exists, then provide path_to_potcar and set this to None 
            path_to_bandoverlaps=os.path.join(directory,"bandOverlaps.lobster.gz"),
            dos_comparison=True, # set to false to disable DOS comparisons 
            bva_comp=True, # set to false to disable LOBSTER charge classification comparisons with BVA method
            path_to_doscar=os.path.join(directory,"DOSCAR.LSO.lobster.gz"),
            e_range=[-20, 0],
            path_to_vasprun=os.path.join(directory,"vasprun.xml.gz"),
            n_bins=256,
        )
calc_quality_K3Sb

In [ ]:
# Get a text description from calculation quality summary dictionary
calc_quality_k3sb_des = Description.get_calc_quality_description(
            calc_quality_K3Sb
        )
Description.write_calc_quality_description(calc_quality_k3sb_des)

## Using plotting utilities

In [ ]:
from matplotlib import style
from pymatgen.io.lobster import Doscar
from lobsterpy.plotting import InteractiveCohpPlotter, PlainCohpPlotter, PlainDosPlotter, get_style_list

You can alter the apperance of the static plots using the style sheet that comes with LobsterPy or use any of the readily available matplotlib style sheets

### Plot COHPs / COBIS / COOPs from Analysis object

It is important the `are_cobis`/`are_coops` args are set to `True` in the plotter depending on type of files you analyzed or want to plot. Here we will keep them false are we are plotting COHPs

In [ ]:
# Using PlainCohpPlotter to get static plots of relevant bonds from Analysis object

style.use(get_style_list()[0]) # Use the LobsterPy style sheet for the generated plots

cohp_plot_static = PlainCohpPlotter(are_cobis=False, are_coops=False)
for plot_label, label_list in analyse.get_site_bond_resolved_labels().items():
    cohp = analyse.chemenv.completecohp.get_summed_cohp_by_label_list(label_list=label_list)
    cohp_plot_static.add_cohp(plot_label, cohp)
cohp_plot_static.get_plot(ylim=[-15,2]);

:::{note}
You can get plots from orbital resolved anaylsis only when `orbital_resolved` arg is set to `True` when initializing Analysis object.
:::

In [ ]:
# Using PlainCohpPlotter to get static plots of relevant orbitals COHPs from Analysis object

style.use('default') # Complete reset the matplotlib figure style
style.use('seaborn-v0_8-ticks') # use exsiting matplotlib style

cohp_plot_static = PlainCohpPlotter()
for plot_label , orb_data in analyse.get_site_orbital_resolved_labels().items():
    for orb, label_list in orb_data.items():
        cohp = analyse.chemenv.completecohp.get_summed_cohp_by_label_and_orbital_list(label_list=label_list, 
                                                                                      orbital_list=[orb]*len(label_list))
        cohp_plot_static.add_cohp(orb, cohp)
cohp_plot_static.get_plot(ylim=[-15,2]);

In [ ]:
# Using interactive plotter to add relevant cohps
interactive_cohp_plot = InteractiveCohpPlotter()

In [ ]:
interactive_cohp_plot.add_all_relevant_cohps(analyse=analyse, label_resolved=False,orbital_resolved=True,suffix='')

In [ ]:
fig = interactive_cohp_plot.get_plot()
fig.show(renderer='notebook')

### Plot DOS from Lobster

In [ ]:
# Load Lobster DOS
dos = Doscar(doscar='../../tests/test_data/NaCl_comp_range/DOSCAR.lobster.gz',
            structure_file='../../tests/test_data/NaCl_comp_range/POSCAR.gz')

Plot total, element and spd dos

In [ ]:
style.use('default') # Complete reset the matplotlib figure style
style.use(get_style_list()[0]) # Use the LobsterPy style sheet for the generated plots

dos_plotter = PlainDosPlotter(summed=True, stack=False, sigma=None)
dos_plotter.add_dos(dos=dos.completedos, label='Total DOS')
dos_plotter.add_dos_dict(dos_dict=dos.completedos.get_element_dos()) # Add element dos
dos_plotter.add_dos_dict(dos_dict=dos.completedos.get_spd_dos()) # add spd dos
dos_plotter.get_plot(xlim=[-10, 3]);

Plotting DOS at particular site and orbital

In [ ]:
dos_plotter = PlainDosPlotter(summed=True, stack=False, sigma=0.03)
dos_plotter.add_site_orbital_dos(dos = dos.completedos, site_index=0, orbital='3s')
dos_plotter.get_plot(xlim=[-10, 3]);

## Use featurizer to extract LOBSTER bonding analysis data as feature for ML studies

:::{note}
To use the batch featurizers, the path to parent directory containing LOBSTER calculation outputs needs to be provided. For example your directory structure needs to be like this: 

parent_dir/lobster_calc_output_dir_for_compound_1/
parent_dir/lobster_calc_output_dir_for_compound_2/
parent_dir/lobster_calc_output_dir_for_compound_3/

the `lobster_calc_output_dir_for_compound_*` directory should contain all your LOBSTER outputs and POSCAR file.

In such a case `path_to_lobster_calcs="parent_dir"` needs to be set 
:::

In [ ]:
from lobsterpy.featurize.batch import BatchSummaryFeaturizer, BatchCoxxFingerprint

`BatchSummaryFeaturizer` provides an convenient way to directly extract summary stats in the form of pandas dataframe directly from LOBSTER calculation directory. The summary stats consist of following:

1. ICOHP, bonding , antibonding percent (mean, min, max , standard deviation) of relevant bonds from LobsterPy analysis (Orbital wise anaylsis stats data can also be included : Optional)
2. Weighted ICOHP ( ICOOP/ ICOBI : Optional)
3. COHP center, width, skewness , kurtosis, edge (COOP/ COBI : Optional)
4. Ionicity and Madelung energies for the structure based on Mulliken and Loewdin charges

In [ ]:
summary_features = BatchSummaryFeaturizer(
            path_to_lobster_calcs="../../tests/test_data/Featurizer_test_data/Lobster_calcs",
            bonds="all",
            include_cobi_data=False,
            include_coop_data=False,
            e_range=[-15, 0],
            n_jobs=3,
        )

In [ ]:
summary_features.get_df()

`BatchCoxxFingerprint` provides an convenient way to directly generate fingerprint objects from COHP / COBI/ COOPCAR.lobster data. Generating fingerprints specificaly for `bonding`, `antibonding` and `overall` interactions is feasible. 

One can also generate a pair-wise fingerprint similarity matrix dataframe (currently only simple vector dot product or tanimoto index are implemented)

In [ ]:
fp_cohp_bonding = BatchCoxxFingerprint(
            path_to_lobster_calcs="../../tests/test_data/Featurizer_test_data/Lobster_calcs",
            e_range=[-15, 0], 
            feature_type="bonding",
            normalize=True, # affects the fingerprint similarity matrix computation
            tanimoto=True, # affects the fingerprint similarity matrix computation
            n_jobs=3,
        fingerprint_for='cohp' # changing this to cobi/coop will result in reading cobicar/coopcar file
        )

In [ ]:
# Access fingerprint dataframe
fp_cohp_bonding.fingerprint_df

In [ ]:
# Get fingerprint similarity matrix
fp_cohp_bonding.get_similarity_matrix_df()

## Generate structure graph objects with LOBSTER data

In [ ]:
from lobsterpy.structuregraph.graph import LobsterGraph

Below code snippet will generate a networkx graph object with ICOHP, ICOOP and ICOBI data as edge properites and charges as node properties.

In [ ]:
graph_NaCl_all = LobsterGraph(
    path_to_poscar="../../tests/test_data/NaCl_comp_range/POSCAR.gz",
    path_to_charge="../../tests/test_data/NaCl_comp_range/CHARGE.lobster.gz",
    path_to_cohpcar="../../tests/test_data/NaCl_comp_range/COHPCAR.lobster.gz",
    path_to_icohplist="../../tests/test_data/NaCl_comp_range/ICOHPLIST.lobster.gz",
    add_additional_data_sg=True,
    path_to_icooplist="../../tests/test_data/NaCl_comp_range/ICOOPLIST.lobster.gz",
    path_to_icobilist="../../tests/test_data/NaCl_comp_range/ICOBILIST.lobster.gz",
    path_to_madelung="../../tests/test_data/NaCl_comp_range/MadelungEnergies.lobster.gz",
    which_bonds="all",
    start=None,
)

In [ ]:
graph_NaCl_all.sg.graph.nodes.data() # view node data

In [ ]:
graph_NaCl_all.sg.graph.edges.data() # view edge data